## Loading

In [4]:
!pip install pandas sklearn gensim tensorflow sentence-transformers
!pip install nltk

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import gensim

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
df = pd.read_csv("cleaned_car_data.csv")
df.reset_index(drop=True, inplace=True)
df.rename(columns={"Unnamed: 0": "id"}, inplace=True)
# df = df[df['Car Brand']=='honda']
df.head()

,id,Car Brand,Car Model,Year,Review,Rating,Value Rating,Performance Rating,Quality Rating,Comfort Rating,...,Vehicle Class,Model Full Name,Horsepower (hp),Curb Weight (lbs),Combined MPG,Fuel Capacity (gallons),Drivetrain,Recommended Fuel,Average MSRP Price ($),Average KBB Fair Price ($)
0,0,honda,accord,2010,I've driven my LX over 13 years and the only t...,5.0,5.0,5.0,5.0,5.0,...,mid-size,Honda Accord,190.0,3327.0,25.0,18.5,FWD,Regular,25117.5,6605.0
1,1,honda,accord,2010,I have this car for 10 years have given me no ...,5.0,5.0,5.0,5.0,5.0,...,mid-size,Honda Accord,190.0,3327.0,25.0,18.5,FWD,Regular,25117.5,6605.0
2,2,honda,accord,2010,"My 2012 Honda Accord is great, It handles grea...",5.0,4.0,5.0,5.0,5.0,...,mid-size,Honda Accord,190.0,3327.0,25.0,18.5,FWD,Regular,25117.5,6605.0
3,3,honda,accord,2010,My sibling has this car. I may be a computer p...,5.0,5.0,5.0,5.0,5.0,...,mid-size,Honda Accord,190.0,3327.0,25.0,18.5,FWD,Regular,25117.5,6605.0
4,4,honda,accord,2010,"I've had this car for 7 years, it's dependable...",5.0,5.0,5.0,5.0,5.0,...,mid-size,Honda Accord,190.0,3327.0,25.0,18.5,FWD,Regular,25117.5,6605.0


In [26]:
# Define the list of stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords
def remove_stopwords(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    words = text.split()  # Split text into words
    cleaned_text = " ".join([word for word in words if word.lower() not in stop_words])  # Remove stopwords
    return cleaned_text

# Apply the function to the Review column
df['Review'] = df['Review'].astype(str).apply(remove_stopwords)

## TF-IDF + NMF


In [81]:
state = 2
topicnum = 10

In [82]:
# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1,3), strip_accents='ascii', max_df=0.5)

# Fit and transform the data
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Review'])
nmf_model = NMF(n_components=topicnum, random_state=state)

# Fit the model
nmf_matrix = nmf_model.fit_transform(tfidf_matrix)  # Document-topic matrix

# Get the top terms per topic
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame to store topics and their contributions for each document
topic_columns = [f"Topic_{i}" for i in range(topicnum)]
nmf_df = pd.DataFrame(nmf_matrix, columns=topic_columns)

# Append topic columns to the original DataFrame
df_nmf = pd.concat([df, nmf_df], axis=1)

# Display the top words for each topic for interpretability
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(", ".join([f"{feature_names[i]} ({topic[i]:.4f})" for i in topic.argsort()[:-no_top_words - 1:-1]]))

# Display top words for each topic
display_topics(nmf_model, feature_names, 10)

Topic 0:
like (0.9768), seats (0.9419), get (0.7246), driving (0.6836), back (0.6321), mpg (0.6206), interior (0.5778), seat (0.5641), system (0.5468), really (0.5419)
Topic 1:
great (3.3377), great car (1.1750), car great (0.3473), comfortable (0.2605), ride (0.2483), looks (0.2421), great gas (0.2158), reliable (0.2049), runs (0.2000), looks great (0.1994)
Topic 2:
good (2.9696), condition (0.5898), good car (0.5614), good condition (0.4962), nice (0.3297), good gas (0.3064), car good (0.2805), tires (0.2336), clean (0.2258), excellent (0.1923)
Topic 3:
miles (1.5132), new (1.1840), bought (0.7339), years (0.7155), oil (0.6978), engine (0.6304), issues (0.6100), transmission (0.6057), never (0.5842), one (0.5460)
Topic 4:
best (1.4385), ever (1.2700), owned (1.1426), ive (0.8515), ever owned (0.8301), best car (0.7137), car ever (0.5774), ive ever (0.5061), best car ever (0.4966), ive ever owned (0.3886)
Topic 5:
truck (2.7721), great truck (0.2918), trucks (0.2058), bed (0.1734), lo

In [83]:
df_nmf.head()

,id,Car Brand,Car Model,Year,Review,Rating,Value Rating,Performance Rating,Quality Rating,Comfort Rating,...,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9
0,0,honda,accord,2010,Ive driven LX years thing Ive replace one whee...,5.0,5.0,5.0,5.0,5.0,...,0.003210,0.026730,0.005335,0.047969,0.029695,0.000000,0.001971,0.000000,0.000000,0.002605
1,1,honda,accord,2010,car years given issues miles still going like ...,5.0,5.0,5.0,5.0,5.0,...,0.000000,0.001019,0.000000,0.068356,0.000000,0.000000,0.000167,0.000000,0.000000,0.000000
2,2,honda,accord,2010,Honda Accord great handles great road st week ...,5.0,4.0,5.0,5.0,5.0,...,0.010233,0.028875,0.001169,0.055385,0.017087,0.000000,0.000000,0.008517,0.000000,0.000000
3,3,honda,accord,2010,sibling car may computer programmer notice goo...,5.0,5.0,5.0,5.0,5.0,...,0.004722,0.000000,0.048911,0.000345,0.000000,0.000000,0.052618,0.000000,0.000000,0.000000
4,4,honda,accord,2010,Ive car years dependable stylish love sun roof...,5.0,5.0,5.0,5.0,5.0,...,0.002605,0.001913,0.002783,0.008760,0.021104,0.001566,0.047558,0.001747,0.001904,0.004344


## LDA

In [84]:
# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1,3), strip_accents='ascii', max_df=0.5)

# Fit and transform the data
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Review'])

# Convert the TF-IDF matrix to a gensim-compatible corpus
tfidf_corpus = gensim.matutils.Sparse2Corpus(tfidf_matrix.T)
feature_names = tfidf_vectorizer.get_feature_names_out()
dictionary = corpora.Dictionary.from_corpus(tfidf_corpus, id2word=dict(enumerate(feature_names)))

# Build the LDA model
lda_model = LdaModel(tfidf_corpus, num_topics=topicnum, id2word=dictionary, passes=10, random_state=state)

# Get the topic distribution for each document and top words per topic
topic_distributions = [lda_model.get_document_topics(doc) for doc in tfidf_corpus]
lda_features = pd.DataFrame([{f'Topic_{i}': score for i, score in doc} for doc in topic_distributions])

# Get top words for each topic and add to DataFrame
topics = lda_model.show_topics(num_topics=topicnum, num_words=10, formatted=False)
top_words = {f'Topic_{i}_words': [word for word, _ in topic] for i, topic in topics}
for topic_name, words in top_words.items():
    lda_features[topic_name] = ", ".join(words)

# Combine with original DataFrame
df_lda = pd.concat([df, lda_features], axis=1)

In [85]:
df_lda.head()

,id,Car Brand,Car Model,Year,Review,Rating,Value Rating,Performance Rating,Quality Rating,Comfort Rating,...,Topic_0_words,Topic_1_words,Topic_2_words,Topic_3_words,Topic_4_words,Topic_5_words,Topic_6_words,Topic_7_words,Topic_8_words,Topic_9_words
0,0,honda,accord,2010,Ive driven LX years thing Ive replace one whee...,5.0,5.0,5.0,5.0,5.0,...,"love, great, best, ever, car ever, love car, b...","ive, volvo, owned, corvette, ive ever, ive own...","like, drive, get, vehicle, driving, love, one,...","miles, new, oil, never, issues, years, bought,...","fun, drive, fun drive, fun car, nice, nice car...","transmission, terrible, transmission problems,...","seats, system, row, seat, room, heated, front,...","recommend, highly, highly recommend, would rec...","great, gas, mileage, gas mileage, good, comfor...","good, condition, clean, good car, low mileage,..."
1,1,honda,accord,2010,car years given issues miles still going like ...,5.0,5.0,5.0,5.0,5.0,...,"love, great, best, ever, car ever, love car, b...","ive, volvo, owned, corvette, ive ever, ive own...","like, drive, get, vehicle, driving, love, one,...","miles, new, oil, never, issues, years, bought,...","fun, drive, fun drive, fun car, nice, nice car...","transmission, terrible, transmission problems,...","seats, system, row, seat, room, heated, front,...","recommend, highly, highly recommend, would rec...","great, gas, mileage, gas mileage, good, comfor...","good, condition, clean, good car, low mileage,..."
2,2,honda,accord,2010,Honda Accord great handles great road st week ...,5.0,4.0,5.0,5.0,5.0,...,"love, great, best, ever, car ever, love car, b...","ive, volvo, owned, corvette, ive ever, ive own...","like, drive, get, vehicle, driving, love, one,...","miles, new, oil, never, issues, years, bought,...","fun, drive, fun drive, fun car, nice, nice car...","transmission, terrible, transmission problems,...","seats, system, row, seat, room, heated, front,...","recommend, highly, highly recommend, would rec...","great, gas, mileage, gas mileage, good, comfor...","good, condition, clean, good car, low mileage,..."
3,3,honda,accord,2010,sibling car may computer programmer notice goo...,5.0,5.0,5.0,5.0,5.0,...,"love, great, best, ever, car ever, love car, b...","ive, volvo, owned, corvette, ive ever, ive own...","like, drive, get, vehicle, driving, love, one,...","miles, new, oil, never, issues, years, bought,...","fun, drive, fun drive, fun car, nice, nice car...","transmission, terrible, transmission problems,...","seats, system, row, seat, room, heated, front,...","recommend, highly, highly recommend, would rec...","great, gas, mileage, gas mileage, good, comfor...","good, condition, clean, good car, low mileage,..."
4,4,honda,accord,2010,Ive car years dependable stylish love sun roof...,5.0,5.0,5.0,5.0,5.0,...,"love, great, best, ever, car ever, love car, b...","ive, volvo, owned, corvette, ive ever, ive own...","like, drive, get, vehicle, driving, love, one,...","miles, new, oil, never, issues, years, bought,...","fun, drive, fun drive, fun car, nice, nice car...","transmission, terrible, transmission problems,...","seats, system, row, seat, room, heated, front,...","recommend, highly, highly recommend, would rec...","great, gas, mileage, gas mileage, good, comfor...","good, condition, clean, good car, low mileage,..."


In [86]:
for i in topics:
  print(i[1])

[('love', 0.029390467), ('great', 0.028955149), ('best', 0.02154447), ('ever', 0.01795618), ('car ever', 0.017639361), ('love car', 0.015451693), ('best car', 0.014086685), ('drives', 0.013906031), ('best car ever', 0.013242207), ('great car', 0.012790388)]
[('ive', 0.025047917), ('volvo', 0.020319873), ('owned', 0.02002493), ('corvette', 0.017985312), ('ive ever', 0.015868656), ('ive owned', 0.013813951), ('ive ever owned', 0.012923116), ('car ive', 0.012227059), ('ever owned', 0.012184579), ('ever', 0.011587757)]
[('like', 0.0056362893), ('drive', 0.0053771036), ('get', 0.005314999), ('vehicle', 0.005235041), ('driving', 0.0048025623), ('love', 0.0047187833), ('one', 0.0046686307), ('great', 0.004485222), ('would', 0.0035968435), ('much', 0.0035930616)]
[('miles', 0.013418172), ('new', 0.012209521), ('oil', 0.008518311), ('never', 0.0070894533), ('issues', 0.0070334678), ('years', 0.006998578), ('bought', 0.0066970196), ('replaced', 0.0062943585), ('service', 0.0060685175), ('engine'

## BERT

In [87]:
# Load pre-trained BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate BERT embeddings for each review
embeddings = model.encode(df['Review'].tolist(), show_progress_bar=True)

# Cluster embeddings using KMeans
num_clusters = topicnum
kmeans = KMeans(n_clusters=num_clusters, random_state=state)
kmeans_labels = kmeans.fit_predict(embeddings)

# Add cluster labels to the DataFrame
df['BERT_Topic'] = kmeans_labels

# Extract top words for each topic using TF-IDF within each cluster
top_words = []
tfidf_vectorizer = TfidfVectorizer(max_features=10, ngram_range=(1, 2), stop_words='english', strip_accents='ascii', max_df = 0.5)

for cluster in range(num_clusters):
    # Select reviews in the current cluster
    cluster_reviews = df[df['BERT_Topic'] == cluster]['Review'].tolist()

    # Generate TF-IDF matrix for the current cluster
    if cluster_reviews:  # Ensure there's at least one review in the cluster
        tfidf_matrix = tfidf_vectorizer.fit_transform(cluster_reviews)
        feature_names = tfidf_vectorizer.get_feature_names_out()

        # Calculate average TF-IDF score for each feature within the cluster
        tfidf_scores = tfidf_matrix.mean(axis=0).A1  # Convert to a 1D array
        top_indices = tfidf_scores.argsort()[-10:][::-1]  # Indices of top words by score
        top_words.append(", ".join([feature_names[i] for i in top_indices]))
    else:
        top_words.append("No reviews")

# Add the top words for each topic to the DataFrame
df['BERT_Topic_Words'] = df['BERT_Topic'].map({i: top_words[i] for i in range(num_clusters)})

# Combine with original DataFrame and embeddings
df_bert = pd.concat([df, pd.DataFrame(embeddings, columns=[f'BERT_{i}' for i in range(embeddings.shape[1])])], axis=1)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/477 [00:00<?, ?it/s]

In [ ]:
df_bert.head()

In [ ]:
df["BERT_Topic_Words"].unique()
# df["BERT_Topic"].unique()

## LSA

In [90]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english', strip_accents='ascii', max_df = 0.5)
X_tfidf = tfidf_vectorizer.fit_transform(df["Review"].tolist())

# Apply LSA using TruncatedSVD
lsa_model = TruncatedSVD(n_components=topicnum, random_state=state)
lsa_features = lsa_model.fit_transform(X_tfidf)

# Get top words and their weights for each topic
feature_names = tfidf_vectorizer.get_feature_names_out()
top_words = {}
for i, topic in enumerate(lsa_model.components_):
    top_word_indices = topic.argsort()[:-11:-1]  # Top 10 words per topic
    top_words[f'LSA_Topic_{i}_words'] = ", ".join([f"{feature_names[j]} ({topic[j]:.4f})" for j in top_word_indices])

# Convert LSA features to DataFrame
lsa_features_df = pd.DataFrame(lsa_features, columns=[f'LSA_Topic_{i}' for i in range(topicnum)])

# Add top words with weights to DataFrame
for topic_name, words in top_words.items():
    lsa_features_df[topic_name] = words

# Combine with original DataFrame
df_lsa = pd.concat([df, lsa_features_df], axis=1)

In [91]:
df_lsa.head()

,id,Car Brand,Car Model,Year,Review,Rating,Value Rating,Performance Rating,Quality Rating,Comfort Rating,...,LSA_Topic_0_words,LSA_Topic_1_words,LSA_Topic_2_words,LSA_Topic_3_words,LSA_Topic_4_words,LSA_Topic_5_words,LSA_Topic_6_words,LSA_Topic_7_words,LSA_Topic_8_words,LSA_Topic_9_words
0,0,honda,accord,2010,Ive driven LX years thing Ive replace one whee...,5.0,5.0,5.0,5.0,5.0,...,"great (0.2599), vehicle (0.1918), good (0.1914...","great (0.5149), good (0.3132), gas (0.1752), m...","good (0.7929), condition (0.1510), mileage (0....","great (0.6138), miles (0.2454), truck (0.1943)...","truck (0.8685), love (0.2939), best (0.0902), ...","vehicle (0.6360), best (0.3754), owned (0.2840...","love (0.7079), good (0.2293), owned (0.1215), ...","vehicle (0.5463), love (0.2715), seats (0.1035...","drive (0.4607), fun (0.2961), vehicle (0.1834)...","drive (0.3987), new (0.3156), fun (0.2431), ve..."
1,1,honda,accord,2010,car years given issues miles still going like ...,5.0,5.0,5.0,5.0,5.0,...,"great (0.2599), vehicle (0.1918), good (0.1914...","great (0.5149), good (0.3132), gas (0.1752), m...","good (0.7929), condition (0.1510), mileage (0....","great (0.6138), miles (0.2454), truck (0.1943)...","truck (0.8685), love (0.2939), best (0.0902), ...","vehicle (0.6360), best (0.3754), owned (0.2840...","love (0.7079), good (0.2293), owned (0.1215), ...","vehicle (0.5463), love (0.2715), seats (0.1035...","drive (0.4607), fun (0.2961), vehicle (0.1834)...","drive (0.3987), new (0.3156), fun (0.2431), ve..."
2,2,honda,accord,2010,Honda Accord great handles great road st week ...,5.0,4.0,5.0,5.0,5.0,...,"great (0.2599), vehicle (0.1918), good (0.1914...","great (0.5149), good (0.3132), gas (0.1752), m...","good (0.7929), condition (0.1510), mileage (0....","great (0.6138), miles (0.2454), truck (0.1943)...","truck (0.8685), love (0.2939), best (0.0902), ...","vehicle (0.6360), best (0.3754), owned (0.2840...","love (0.7079), good (0.2293), owned (0.1215), ...","vehicle (0.5463), love (0.2715), seats (0.1035...","drive (0.4607), fun (0.2961), vehicle (0.1834)...","drive (0.3987), new (0.3156), fun (0.2431), ve..."
3,3,honda,accord,2010,sibling car may computer programmer notice goo...,5.0,5.0,5.0,5.0,5.0,...,"great (0.2599), vehicle (0.1918), good (0.1914...","great (0.5149), good (0.3132), gas (0.1752), m...","good (0.7929), condition (0.1510), mileage (0....","great (0.6138), miles (0.2454), truck (0.1943)...","truck (0.8685), love (0.2939), best (0.0902), ...","vehicle (0.6360), best (0.3754), owned (0.2840...","love (0.7079), good (0.2293), owned (0.1215), ...","vehicle (0.5463), love (0.2715), seats (0.1035...","drive (0.4607), fun (0.2961), vehicle (0.1834)...","drive (0.3987), new (0.3156), fun (0.2431), ve..."
4,4,honda,accord,2010,Ive car years dependable stylish love sun roof...,5.0,5.0,5.0,5.0,5.0,...,"great (0.2599), vehicle (0.1918), good (0.1914...","great (0.5149), good (0.3132), gas (0.1752), m...","good (0.7929), condition (0.1510), mileage (0....","great (0.6138), miles (0.2454), truck (0.1943)...","truck (0.8685), love (0.2939), best (0.0902), ...","vehicle (0.6360), best (0.3754), owned (0.2840...","love (0.7079), good (0.2293), owned (0.1215), ...","vehicle (0.5463), love (0.2715), seats (0.1035...","drive (0.4607), fun (0.2961), vehicle (0.1834)...","drive (0.3987), new (0.3156), fun (0.2431), ve..."


In [92]:
print(top_words.items())

dict_items([('LSA_Topic_0_words', 'great (0.2599), vehicle (0.1918), good (0.1914), miles (0.1745), love (0.1596), drive (0.1555), like (0.1554), new (0.1454), gas (0.1318), comfortable (0.1284)'), ('LSA_Topic_1_words', 'great (0.5149), good (0.3132), gas (0.1752), mileage (0.1533), comfortable (0.1455), ride (0.0960), love (0.0950), nice (0.0850), seats (0.0822), fun (0.0642)'), ('LSA_Topic_2_words', 'good (0.7929), condition (0.1510), mileage (0.0889), tires (0.0862), new (0.0743), miles (0.0708), nice (0.0706), gas (0.0705), oil (0.0653), truck (0.0645)'), ('LSA_Topic_3_words', 'great (0.6138), miles (0.2454), truck (0.1943), new (0.1846), oil (0.1236), problems (0.1088), bought (0.0925), runs (0.0915), tires (0.0893), maintenance (0.0885)'), ('LSA_Topic_4_words', 'truck (0.8685), love (0.2939), best (0.0902), owned (0.0785), ive (0.0711), trucks (0.0641), bed (0.0500), cab (0.0438), ford (0.0403), gmc (0.0387)'), ('LSA_Topic_5_words', 'vehicle (0.6360), best (0.3754), owned (0.2840